# Medallion Architecture

**Cel szkoleniowy:** Opanowanie implementacji architektury medalionowej (Bronze → Silver → Gold) w Databricks.

**Zakres tematyczny:**
- Bronze Layer: Ingestion surowych danych (CSV, JSON, Parquet)
- Silver Layer: Data Quality, Deduplication & Validation
- SCD (Slowly Changing Dimensions): Implementacja SCD Type 1 i Type 2
- Gold Layer: Agregacje biznesowe i Star Schema
- Best Practices: Partycjonowanie, Z-Ordering, Data Retention

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 2 - Delta Lake & Lakehouse
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
  - Databricks Runtime 16.4 LTS lub nowszy (zalecane: 17.3 LTS)
  - Unity Catalog włączony
  - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Klaster: Standard lub **Serverless Compute** (zalecane)
- **Zależności**: 
  - Wykonany notebook `01_delta_lake_operations.ipynb`
  - Wykonany notebook `02_Lakeflow_Connection.ipynb` (dla danych Bronze)
- **Czas realizacji**: ~90 minut

> **Uwaga (2025):** Serverless Compute jest teraz domyślnym trybem dla nowych workloadów.

## Wstęp teoretyczny - Medallion Architecture

**Cel sekcji:** Zrozumienie architektury medalionowej jako fundamentalnego design pattern dla data lakehouse.

---

### Czym jest Medallion Architecture?

**Medallion Architecture** to wielowarstwowy wzorzec organizacji danych w data lakehouse, który dzieli dane na trzy warstwy o rosnącej jakości i wartości biznesowej:

```
DATA SOURCES
    ↓
🥉 BRONZE (Raw)
    ↓ cleansing
🥈 SILVER (Validated)
    ↓ aggregation
🥇 GOLD (Business)
    ↓
CONSUMPTION
```

### Warstwy - Szczegółowy Opis

#### 🥉 Bronze Layer - Raw / Landing Zone

**Charakterystyka:**
- Dane "as-is" bez transformacji wartości
- Append-only, immutable
- Audit metadata: `_ingestion_timestamp`, `_source_file`, `_user`
- Multi-format: JSON, CSV, Parquet, Avro
- Schema-on-read approach

**Retention:** 3-7 lat (długoterminowa historia)

**Use Cases:**
- Data recovery (reprocess pipeline)
- Audit trail & compliance
- Historical analysis
- Data science exploration

**Przykład Bronze Table:**
```sql
CREATE TABLE bronze.orders_raw (
    order_id STRING,
    customer_id STRING,
    order_date STRING,        -- Raw string, nie parsed
    total_amount STRING,      -- Raw string, nie validated
    payment_method STRING,
    _ingestion_timestamp TIMESTAMP,
    _source_file STRING,
    _rescued_data STRING      -- Schema evolution
)
```

---

#### 🥈 Silver Layer - Cleansed / Validated

**Charakterystyka:**
- **Deduplikacja** po kluczu biznesowym
- **Walidacja**: NOT NULL, data types, ranges
- **Standaryzacja**: dates, text, formats
- **Business rules** enforcement
- **Schema enforcement** (strict schema)
- **Upsert/Merge** patterns (SCD)

**Retention:** 1-2 lata (medium-term history)

**Use Cases:**
- Foundation for analytics
- Joins & enrichment
- ML feature engineering
- Data quality monitoring

**Przykład Silver Table:**
```sql
CREATE TABLE silver.orders_clean (
    order_id BIGINT NOT NULL,     -- Validated, parsed
    customer_id BIGINT NOT NULL,
    order_date DATE NOT NULL,     -- Parsed to DATE
    total_amount DECIMAL(10,2),   -- Validated numeric
    payment_method STRING,
    _quality_score INT,           -- Data quality metric
    _processing_timestamp TIMESTAMP,
    _is_valid BOOLEAN
)
```

---

#### 🥇 Gold Layer - Business / Aggregates

**Charakterystyka:**
- **Pre-aggregated** summaries (daily, monthly, yearly)
- **Denormalized** tables (joins pre-computed)
- **KPI calculations** & business metrics
- **Star schema** / dimensional models
- **ML feature stores**
- **Query-optimized** (partitioned, indexed)

**Retention:** 6-12 miesięcy (short-term, refreshable)

**Use Cases:**
- BI dashboards (Power BI, Tableau)
- Executive reports
- ML model training
- Self-service analytics

**Przykład Gold Table:**
```sql
CREATE TABLE gold.daily_sales_summary (
    report_date DATE NOT NULL,
    payment_method STRING,
    total_orders BIGINT,
    total_revenue DECIMAL(15,2),
    avg_order_value DECIMAL(10,2),
    unique_customers BIGINT,
    _computation_timestamp TIMESTAMP
)
PARTITIONED BY (report_date)
```

---

### Kluczowe Zasady Medallion Architecture

**1. Separation of Concerns**
- Bronze: Ingestion
- Silver: Data Quality
- Gold: Business Logic

**2. Incremental Processing**
- Process tylko nowe/zmienione dane
- Delta Lake MERGE operations
- Checkpoint management

**3. Idempotency**
- Można uruchomić wielokrotnie bez duplikacji
- Deterministic transformations
- Unique keys & deduplication

**4. Schema Evolution**
- Bronze: Flexible (rescued data)
- Silver: Controlled (addNewColumns)
- Gold: Strict (versioned)

**5. Data Quality Gates**
- Validate before promoting to next layer
- Quarantine bad records
- Monitoring & alerting

---

### ETL vs ELT w Medallion

**Traditional ETL:**
```
Extract → Transform → Load
         (outside DB)
```

**Medallion ELT:**
```
Extract → Load (Bronze) → Transform (Silver) → Load (Gold)
                 ↓                    ↓              ↓
             raw data          cleansed data    aggregates
```

**Dlaczego ELT?**
- Zachowanie raw data (compliance)
- Flexibility (re-transform later)
- Scalability (Spark distributed processing)
- Cost-effective (storage cheaper than compute)

---

### Medallion vs Traditional Data Warehouse

| Feature | Traditional DWH | Medallion Lakehouse |
|---------|-----------------|---------------------|
| **Storage** | Proprietary (expensive) | Cloud object storage (cheap) |
| **Schema** | Schema-on-write | Schema-on-read (Bronze) |
| **Data Types** | Structured only | Structured + semi-structured |
| **Flexibility** | Rigid | Flexible (schema evolution) |
| **Raw Data** | Discarded | Preserved (Bronze) |
| **Processing** | ETL (batch) | ELT (batch + streaming) |
| **Cost** | High (compute + storage) | Lower (decouple compute/storage) |
| **Use Cases** | BI & reporting | BI + ML + data science |

---

### Best Practices

**1. Naming Conventions:**
```
bronze.{source_system}_{entity}_raw
silver.{entity}_clean
gold.{business_domain}_{aggregation_level}
```

**2. Partitioning:**
- Bronze: Ingestion date (`_ingestion_date`)
- Silver: Business date (`order_date`, `transaction_date`)
- Gold: Report date (`report_date`)

**3. Metadata Columns:**
```python
# Bronze
_ingestion_timestamp, _source_file, _user

# Silver
_processing_timestamp, _quality_score, _is_valid

# Gold
_computation_timestamp, _version
```

**4. Refresh Cadence:**
- Bronze: Real-time / hourly
- Silver: Hourly / daily
- Gold: Daily / on-demand

**5. Data Retention:**
```python
# Bronze: 3-7 years (compliance)
spark.sql("ALTER TABLE bronze.orders SET TBLPROPERTIES (
    'delta.logRetentionDuration' = '2555 days',
    'delta.deletedFileRetentionDuration' = '2555 days'
)")

# Silver: 1-2 years
# Gold: 6-12 months (refreshable from Silver)
```

---

### Pipeline Architecture Diagram

```
┌──────────────────────────────────────────────────────────────┐
│                      DATA SOURCES                             │
│  • PostgreSQL  • MySQL  • APIs  • S3 Files  • Kafka          │
└────────────────────────┬─────────────────────────────────────┘
                         │ COPY INTO / Auto Loader
                         ▼
┌──────────────────────────────────────────────────────────────┐
│                   🥉 BRONZE LAYER                             │
│                                                               │
│  bronze.customers_raw      bronze.orders_raw                 │
│  bronze.products_raw       bronze.events_raw                 │
│                                                               │
│  Features:                                                    │
│  • Raw data (as-is)                                          │
│  • Append-only                                               │
│  • Audit metadata                                            │
│  • Long retention (3-7y)                                     │
└────────────────────────┬─────────────────────────────────────┘
                         │ MERGE (Dedup, Validate)
                         ▼
┌──────────────────────────────────────────────────────────────┐
│                   🥈 SILVER LAYER                             │
│                                                               │
│  silver.customers_clean    silver.orders_clean               │
│  silver.products_clean     silver.events_clean               │
│                                                               │
│  Features:                                                    │
│  • Deduplicated                                              │
│  • Validated (types, nulls)                                  │
│  • SCD Type 1/2 (history)                                    │
│  • Medium retention (1-2y)                                   │
└────────────────────────┬─────────────────────────────────────┘
                         │ GROUP BY / JOIN / AGGREGATE
                         ▼
┌──────────────────────────────────────────────────────────────┐
│                   🥇 GOLD LAYER                               │
│                                                               │
│  gold.daily_sales_summary                                    │
│  gold.customer_360                                           │
│  gold.product_performance                                    │
│                                                               │
│  Features:                                                    │
│  • Pre-aggregated                                            │
│  • Denormalized (star schema)                                │
│  • KPIs & metrics                                            │
│  • Short retention (6-12m)                                   │
└────────────────────────┬─────────────────────────────────────┘
                         │
                         ▼
┌──────────────────────────────────────────────────────────────┐
│                  CONSUMPTION LAYER                            │
│  • Power BI  • Tableau  • SQL Analytics  • ML Models         │
└──────────────────────────────────────────────────────────────┘
```

## Izolacja per użytkownik

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [0]:
%run ../00_setup

## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime, timedelta
import time

# Wyświetl kontekst użytkownika
display({
    "Katalog": CATALOG,
    "Schema Bronze": BRONZE_SCHEMA,
    "Schema Silver": SILVER_SCHEMA,
    "Schema Gold": GOLD_SCHEMA,
    "Użytkownik": raw_user,
    "Dataset base": DATASET_BASE_PATH
})

# Ustaw katalog i schemat jako domyślne
spark.sql(f"USE CATALOG {CATALOG}")

### Kontekst użytkownika

Wyświetlenie aktualnej konfiguracji środowiska oraz ścieżek do danych:

In [ ]:
spark.sql(f"USE CATALOG {CATALOG}")

**Konfiguracja katalogu Unity Catalog:**

Ustawienie domyślnego katalogu dla wszystkich operacji.

## Sekcja 1: Bronze Layer - Raw Data Landing

**Cel sekcji:** Zrozumienie roli Bronze layer jako landing zone dla raw data.

### Bronze Layer - Kluczowe Cechy

**1. Raw Data "As-Is"**
- Dane zapisywane bez transformacji wartości
- Zachowanie oryginalnego formatu
- Multi-format support (JSON, CSV, Parquet)

**2. Append-Only Pattern**
- Nigdy nie usuwamy/modyfikujemy danych
- Immutable history
- Time-travel capability

**3. Audit Metadata**
```python
# Metadane audytowe w Bronze
_ingestion_timestamp  # Kiedy załadowano
_source_file         # Skąd pochodzą dane
_user                # Kto załadował
_rescued_data        # Schema evolution (unexpected columns)
```

**4. Schema-on-Read**
- Elastyczny schemat (może się zmieniać)
- Rescued data column dla unknown columns
- Reprocessing capability

### Bronze Tables - Struktura

W tym demo załadujemy dane bezpośrednio z plików źródłowych (CSV, JSON, Parquet) do warstwy Bronze, aby zapewnić niezależność tego notebooka.

**Tabele Bronze:**
- `bronze.customers_raw` - dane klientów (CSV)
- `bronze.orders_raw` - zamówienia (JSON)
- `bronze.products_raw` - produkty (Parquet)

### Dlaczego Bronze jest Ważny?

**1. Data Recovery**
```python
# Możemy reprocessować pipeline od Bronze
bronze_data = spark.table("bronze.orders_raw")
# Re-run transformations → Silver → Gold
```

**2. Schema Evolution**
```python
# Nowe kolumny w source nie łamią pipeline
# Trafiają do _rescued_data
```

**3. Compliance & Audit**
```python
# Pełna historia: kto, co, kiedy załadował
# Retention: 3-7 lat (regulacje prawne)
```

**4. Data Science Exploration**
```python
# Analitycy mogą eksplorować raw data
# Tworzyć nowe features z surowych danych
```

### Przykład 1.1: Inspekcja Bronze Layer

**Cel:** Sprawdzić dane w Bronze layer i zrozumieć ich strukturę.

### Przykład 1.1: Ingestion do Bronze Layer (Raw Data)

Wczytujemy dane bezpośrednio z plików źródłowych (CSV, JSON, Parquet) do tabel Bronze.
Dzięki temu notebook jest niezależny od poprzednich kroków.

In [ ]:
# 1. Load Customers (CSV)
customers_path = f"{DATASET_BASE_PATH}/customers/customers.csv"

customers_df = (spark.read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true") # In Bronze we often infer or use string
    .load(customers_path)
)

display(customers_df.limit(5))

In [ ]:
# 3. Load Products (Parquet)
products_path = f"{DATASET_BASE_PATH}/products/products.parquet"

products_df = (spark.read
    .format("parquet")
    .load(products_path)
)

# Write to Bronze
(products_df.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("bronze.products_raw")
)

display({"status": "✅ Table bronze.products_raw created/overwritten"})

In [ ]:
# 2. Load Orders (JSON)
orders_path = f"{DATASET_BASE_PATH}/orders/orders_batch.json"

orders_df = (spark.read
    .format("json")
    .load(orders_path)
)

# Write to Bronze
(orders_df.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("bronze.orders_raw")
)

display({"status": "✅ Table bronze.orders_raw created/overwritten"})

In [ ]:
# Write to Bronze (Overwrite for full load simulation)
(customers_df.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("bronze.customers_raw")
)

display({"status": "✅ Table bronze.customers_raw created/overwritten"})

**Utworzone tabele Bronze:**

Konwersja danych z batch do raw tables została zakończona pomyślnie.

In [0]:
# Inspekcja Bronze Layer
bronze_tables = ["customers_raw", "orders_raw", "products_raw"]
results = []

for table in bronze_tables:
    full_table = f"{CATALOG}.{BRONZE_SCHEMA}.{table}"
    
    if spark.catalog.tableExists(full_table):
        df = spark.table(full_table)
        results.append({
            "table": table,
            "status": "✅",
            "records": df.count(),
            "columns": len(df.columns)
        })

display(spark.createDataFrame(results))

**Podsumowanie inspekcji Bronze Layer:**

Bronze Layer zawiera RAW data bez transformacji, zachowuje pełną historię (append-only) i stanowi foundation dla dalszych transformacji.

## Sekcja 2: Silver Layer - Cleansing & Validation

**Cel sekcji:** Transformacja danych z Bronze do Silver z zastosowaniem data quality rules.

### Silver Layer - Kluczowe Cechy

**1. Data Cleansing**
- Parsing: string → proper types (INT, DATE, DECIMAL)
- Trimming: whitespace, special characters
- Standardization: dates, phone numbers, emails
- Null handling: replacements, defaults

**2. Deduplication**
- Identyfikacja unique business key
- MERGE operation (upsert pattern)
- Keeping latest version based on timestamp

**3. Validation Rules**
```python
# Przykładowe validations
- NOT NULL constraints
- Range checks (amount > 0)
- Referential integrity (FK exists)
- Business rules (discount <= price)
```

**4. Schema Enforcement**
- Strict schema (vs Bronze flexible)
- Explicit data types
- Column constraints

### Bronze → Silver Transformation Pattern

**Typical Flow:**
```python
bronze_df = spark.table("bronze.orders_raw")

silver_df = (bronze_df
    # 1. Parse & Cast
    .withColumn("order_id", col("order_id").cast("bigint"))
    .withColumn("order_date", to_date(col("order_date")))
    .withColumn("total_amount", col("total_amount").cast("decimal(10,2)"))
    
    # 2. Validate
    .filter(col("order_id").isNotNull())
    .filter(col("total_amount") > 0)
    
    # 3. Standardize
    .withColumn("payment_method", upper(trim(col("payment_method"))))
    
    # 4. Add metadata
    .withColumn("_processing_timestamp", current_timestamp())
    .withColumn("_is_valid", lit(True))
)

# 5. MERGE to Silver (deduplication)
silver_df.write.format("delta") \
    .mode("append") \
    .option("mergeSchema", "false") \
    .saveAsTable("silver.orders_clean")
```

### MERGE Operation - Deduplication Pattern

**Problem:** Bronze zawiera duplikaty (append-only)

**Rozwiązanie:** MERGE w Silver (upsert)

```sql
MERGE INTO silver.orders_clean AS target
USING (
    SELECT DISTINCT *
    FROM bronze.orders_raw
    WHERE _ingestion_timestamp > (
        SELECT MAX(_processing_timestamp)
        FROM silver.orders_clean
    )
) AS source
ON target.order_id = source.order_id
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *
```

### Data Quality Checks

**Levels of Quality:**

**Level 1: Schema Validation**
- Correct data types
- Required columns present
- No unexpected nulls

**Level 2: Business Rules**
- Ranges (amount between 0-1000000)
- Referential integrity (customer_id exists)
- Logical consistency (order_date <= ship_date)

**Level 3: Statistical Checks**
- Outlier detection
- Distribution monitoring
- Anomaly alerts

**Quarantine Pattern:**
```python
# Valid records → Silver
valid_df = df.filter(col("_is_valid") == True)
valid_df.write.saveAsTable("silver.orders_clean")

# Invalid records → Quarantine
invalid_df = df.filter(col("_is_valid") == False)
invalid_df.write.saveAsTable("silver.orders_quarantine")
```

### Przykład 2.1: Bronze → Silver Transformation (Orders)

**Cel:** Transform orders z Bronze do Silver z cleansing i validation.

In [0]:
# Bronze → Silver: Orders
bronze_orders = spark.table(f"{BRONZE_SCHEMA}.orders_raw")

# Transform & Validate
silver_orders = (bronze_orders
    .withColumn("order_id", F.col("order_id").cast("bigint"))
    .withColumn("customer_id", F.col("customer_id").cast("bigint"))
    .withColumn("order_datetime", F.to_timestamp(F.col("order_datetime"), "yyyy-MM-dd HH:mm:ss"))
    .withColumn("total_amount", F.col("total_amount").cast("decimal(10,2)"))
    .withColumn("payment_method", F.upper(F.trim(F.col("payment_method"))))
    .withColumn("_is_valid", 
        F.when(
            (F.col("order_id").isNotNull()) &
            (F.col("customer_id").isNotNull()) &
            (F.col("order_datetime").isNotNull()) &
            (F.col("total_amount") > 0),
            True
        ).otherwise(False)
    )
    .withColumn("_processing_timestamp", F.current_timestamp())
    .select(
        "order_id", "customer_id", "order_datetime", 
        "total_amount", "payment_method",
        "_is_valid", "_processing_timestamp"
    )
)

# Split valid/invalid
valid_orders = silver_orders.filter(F.col("_is_valid") == True)
invalid_orders = silver_orders.filter(F.col("_is_valid") == False)

# Write to Silver
valid_orders.write.format("delta").mode("overwrite").saveAsTable(f"{SILVER_SCHEMA}.orders_clean")

display({
    "bronze_records": bronze_orders.count(),
    "valid_orders": valid_orders.count(),
    "invalid_orders": invalid_orders.count(),
    "status": f"✅ Created {SILVER_SCHEMA}.orders_clean"
})

In [ ]:
display(spark.table(f"{SILVER_SCHEMA}.orders_clean").limit(10))

**Uwaga dotycząca production:**

W środowisku produkcyjnym użylibyśmy operacji MERGE dla deduplikacji zamiast prostego overwrite.

## Sekcja 3: SCD (Slowly Changing Dimensions)

**Cel sekcji:** Implementacja SCD Type 1 i Type 2 dla śledzenia zmian w danych.

### Co to jest SCD?

**Slowly Changing Dimensions (SCD)** to techniki śledzenia zmian w wymiarach (dimension tables) w hurtowniach danych.

**Problem:**
```
Klient zmienia adres:
- Jan Kowalski, Warszawa → Kraków

Pytanie: Czy zachować historię?
```

### SCD Types - Overview

| Type | Strategy | History | Use Case |
|------|----------|---------|----------|
| **Type 0** | No changes allowed | N/A | Reference data (countries) |
| **Type 1** | Overwrite | ❌ No | Current state only |
| **Type 2** | Add new row | ✅ Yes | Full history tracking |
| **Type 3** | Add new column | ⚠️ Limited | Previous value only |

---

### SCD Type 1 - Overwrite

**Strategia:** Nadpisz starą wartość nową (bez historii)

**Implementacja:** Simple UPDATE/MERGE

**Example:**
```
Before:
customer_id | name        | city
1           | Jan Kowalski| Warszawa

Change: city → Kraków

After:
customer_id | name        | city
1           | Jan Kowalski| Kraków     # Overwritten!
```

**Kod SQL:**
```sql
MERGE INTO silver.customers_dim AS target
USING updates AS source
ON target.customer_id = source.customer_id
WHEN MATCHED THEN UPDATE SET
    target.city = source.city,
    target.updated_at = current_timestamp()
```

**Pros:**
- ✅ Simple implementation
- ✅ No history bloat
- ✅ Always current values

**Cons:**
- ❌ No historical tracking
- ❌ Can't analyze "as of date"
- ❌ Lose audit trail

**Use Cases:**
- Correcting data entry errors
- Non-critical attributes (e.g., marketing preferences)
- Reference data that shouldn't have history

---

### SCD Type 2 - Historical Tracking

**Strategia:** Dodaj nowy rekord dla każdej zmiany (pełna historia)

**Implementacja:** MERGE z version tracking

**Example:**
```
Before:
customer_id | name        | city    | effective_from | effective_to | is_current
1           | Jan Kowalski| Warszawa| 2023-01-01     | 9999-12-31   | true

Change: city → Kraków (2024-06-15)

After:
customer_id | name        | city    | effective_from | effective_to | is_current
1           | Jan Kowalski| Warszawa| 2023-01-01     | 2024-06-14   | false  # Closed
1           | Jan Kowalski| Kraków  | 2024-06-15     | 9999-12-31   | true   # New!
```

**Kolumny SCD Type 2:**
- `effective_from` / `valid_from`: Start date
- `effective_to` / `valid_to`: End date (9999-12-31 = current)
- `is_current` / `is_active`: Boolean flag
- `version`: Optional version number
- `surrogate_key`: Technical key (not business key)

**Kod SQL (simplified):**
```sql
-- Step 1: Close old records
UPDATE silver.customers_dim
SET 
    effective_to = current_date() - 1,
    is_current = false
WHERE customer_id IN (SELECT customer_id FROM updates)
  AND is_current = true;

-- Step 2: Insert new records
INSERT INTO silver.customers_dim
SELECT 
    customer_id,
    name,
    city,
    current_date() AS effective_from,
    DATE '9999-12-31' AS effective_to,
    true AS is_current
FROM updates;
```

**Pros:**
- ✅ Full history preserved
- ✅ "As of date" queries possible
- ✅ Audit trail
- ✅ Temporal analytics

**Cons:**
- ❌ Table grows (more rows)
- ❌ More complex queries (need to filter is_current)
- ❌ Surrogate keys needed

**Use Cases:**
- Customer dimensions (address, preferences)
- Product dimensions (price history)
- Employee dimensions (salary, department)
- Compliance & audit requirements

---

### SCD Type 3 - Limited History (rzadziej używany)

**Strategia:** Dodaj kolumnę dla previous value

**Example:**
```
customer_id | name        | city    | previous_city
1           | Jan Kowalski| Kraków  | Warszawa
```

**Pros:**
- ✅ Simple (one previous value)
- ✅ No row explosion

**Cons:**
- ❌ Only 1 previous value
- ❌ Limited analytics

**Use Case:** Rarely used (SCD Type 2 is better)

---

### SCD Decision Matrix

**Kiedy używać którego typu?**

| Requirement | Recommended Type |
|-------------|------------------|
| No history needed | Type 1 |
| Full history required | Type 2 |
| Audit/compliance | Type 2 |
| Data corrections | Type 1 |
| Current state only | Type 1 |
| Temporal analytics | Type 2 |
| Growing table OK | Type 2 |
| Storage constrained | Type 1 |

**💡 Best Practice:** 
- Use **Type 1** for Silver layer (current state)
- Use **Type 2** for Gold dimensional tables (history)

---

### MERGE Pattern for SCD Type 2 (Advanced)

**Complete Implementation:**

```sql
MERGE INTO silver.customers_dim AS target
USING (
    SELECT 
        customer_id,
        name,
        city,
        email,
        current_timestamp() AS effective_from,
        CAST('9999-12-31' AS DATE) AS effective_to,
        true AS is_current
    FROM staging.customers_updates
) AS source
ON target.customer_id = source.customer_id 
   AND target.is_current = true

-- Case 1: No change → do nothing
WHEN MATCHED AND (
    target.city = source.city AND
    target.email = source.email
) THEN UPDATE SET target.updated_at = current_timestamp()

-- Case 2: Change detected → close old, insert new
WHEN MATCHED AND (
    target.city != source.city OR
    target.email != source.email
) THEN UPDATE SET
    target.effective_to = current_date() - 1,
    target.is_current = false

-- Case 3: New customer → insert
WHEN NOT MATCHED THEN INSERT (
    customer_id, name, city, email,
    effective_from, effective_to, is_current
) VALUES (
    source.customer_id, source.name, source.city, source.email,
    source.effective_from, source.effective_to, source.is_current
);

-- Step 2: Insert new versions for changed records
INSERT INTO silver.customers_dim
SELECT 
    source.*
FROM staging.customers_updates AS source
INNER JOIN silver.customers_dim AS target
    ON source.customer_id = target.customer_id
WHERE target.is_current = false
  AND target.effective_to = current_date() - 1;
```

### Przykład 3.1: SCD Type 1 - Customers (Overwrite)

**Cel:** Implementacja SCD Type 1 - prosty overwrite bez historii.

In [0]:
# SCD Type 1: Customers
bronze_customers = spark.table(f"{BRONZE_SCHEMA}.customers_raw")

# Transform to Silver (SCD Type 1 - current state only)
customers_type1 = (bronze_customers
    .withColumn("customer_id", F.col("customer_id").cast("bigint"))
    .withColumn("name", F.trim(F.col("name")))
    .withColumn("email", F.lower(F.trim(F.col("email"))))
    .withColumn("city", F.initcap(F.trim(F.col("city"))))
    .withColumn("updated_at", F.current_timestamp())
    .select("customer_id", "name", "email", "city", "updated_at")
)

# Create/Replace table (Type 1 = overwrite)
customers_type1.write.format("delta").mode("overwrite").saveAsTable(f"{SILVER_SCHEMA}.customers_type1")

display({
    "status": f"✅ Created {SILVER_SCHEMA}.customers_type1",
    "records": customers_type1.count(),
    "note": "SCD Type 1: Zawsze aktualny stan, bez historii"
})

**Utworzona tabela SCD Type 1:**

Tabela `customers_type1` zawiera zawsze aktualny stan bez historii. Wyświetlenie przykładowych danych:

In [ ]:
display(spark.table(f"{SILVER_SCHEMA}.customers_type1").limit(5))

### Symulacja UPDATE - zmiana miasta dla customer_id=1

**PRZED ZMIANĄ:** Wyświetlenie aktualnego stanu customer_id=1:

In [ ]:
spark.sql(f"""
    SELECT * FROM {SILVER_SCHEMA}.customers_type1 
    WHERE customer_id = 1
""").show()

In [ ]:
from pyspark.sql.types import StructType, StructField, LongType, StringType
updates_schema = StructType([
    StructField("customer_id", LongType(), True),
    StructField("name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("city", StringType(), True)
])

updates_data = [(1, "Jan Kowalski", "jan@example.com", "Kraków")]  # Changed city!
updates_df = spark.createDataFrame(updates_data, updates_schema)

In [ ]:
updates_df.createOrReplaceTempView("customer_updates")

spark.sql(f"""
    MERGE INTO {SILVER_SCHEMA}.customers_type1 AS target
    USING customer_updates AS source
    ON target.customer_id = source.customer_id
    WHEN MATCHED THEN UPDATE SET
        target.city = source.city,
        target.updated_at = current_timestamp()
    WHEN NOT MATCHED THEN INSERT *
""")

**PO ZMIANIE (SCD Type 1 - overwrite):** Wyświetlenie zaktualizowanego stanu:

In [ ]:
spark.sql(f"""
    SELECT * FROM {SILVER_SCHEMA}.customers_type1 
    WHERE customer_id = 1
""").show()

**⚠️  UWAGA:** Historia zmiany została UTRACONA

**💡 Stara wartość (Warszawa) została nadpisana (Kraków)**

### Przykład 3.2: SCD Type 2 - Customers (Historical Tracking)

**Cel:** Implementacja SCD Type 2 - pełne śledzenie historii zmian.

In [0]:
# SCD Type 2: Customers (Historical Tracking)
bronze_customers = spark.table(f"{BRONZE_SCHEMA}.customers_raw")

customers_type2_initial = (bronze_customers
    .withColumn("customer_id", F.col("customer_id").cast("bigint"))
    .withColumn("name", F.trim(F.col("name")))
    .withColumn("email", F.lower(F.trim(F.col("email"))))
    .withColumn("city", F.initcap(F.trim(F.col("city"))))
    # SCD Type 2 columns
    .withColumn("effective_from", F.current_date())
    .withColumn("effective_to", F.lit("9999-12-31").cast("date"))
    .withColumn("is_current", F.lit(True))
    .withColumn("version", F.lit(1))
    .select(
        "customer_id", "name", "email", "city",
        "effective_from", "effective_to", "is_current", "version"
    )
)

# Create initial table
customers_type2_initial.write.format("delta").mode("overwrite").saveAsTable(f"{SILVER_SCHEMA}.customers_type2")

display({
    "status": f"✅ Created {SILVER_SCHEMA}.customers_type2",
    "records": customers_type2_initial.count(),
    "columns": "effective_from, effective_to, is_current, version"
})

**Utworzona tabela SCD Type 2:**

Tabela `customers_type2` zawiera kolumny: effective_from, effective_to, is_current, version. Wyświetlenie przykładowych danych:

In [ ]:
display(spark.table(f"{SILVER_SCHEMA}.customers_type2").limit(5))

### Symulacja CHANGE - zmiana miasta dla customer_id=1

**PRZED ZMIANĄ:** Wyświetlenie aktualnego stanu customer_id=1:

In [ ]:
spark.sql(f"""
    SELECT * FROM {SILVER_SCHEMA}.customers_type2 
    WHERE customer_id = 1
    ORDER BY effective_from
""").show()

In [ ]:
updates_data = [(1, "Jan Kowalski", "jan@example.com", "Kraków")]  # Changed city!
updates_df = spark.createDataFrame(updates_data, ["customer_id", "name", "email", "city"])
updates_df.createOrReplaceTempView("customer_updates_type2")

**Step 1: Close old records** (set effective_to, is_current=false):

In [ ]:
spark.sql(f"""
    MERGE INTO {SILVER_SCHEMA}.customers_type2 AS target
    USING (
        SELECT DISTINCT u.customer_id
        FROM customer_updates_type2 u
        INNER JOIN {SILVER_SCHEMA}.customers_type2 t
            ON u.customer_id = t.customer_id
        WHERE t.is_current = true
          AND (u.city != t.city OR u.email != t.email)  -- Detect changes
    ) AS changed
    ON target.customer_id = changed.customer_id 
       AND target.is_current = true
    WHEN MATCHED THEN UPDATE SET
        target.effective_to = current_date() - INTERVAL 1 DAY,
        target.is_current = false
""")

**Step 2: Insert new versions** - dodanie nowych rekordów z updated values:

In [ ]:
spark.sql(f"""
    INSERT INTO {SILVER_SCHEMA}.customers_type2
    SELECT 
        u.customer_id,
        u.name,
        u.email,
        u.city,
        current_date() AS effective_from,
        CAST('9999-12-31' AS DATE) AS effective_to,
        true AS is_current,
        COALESCE(MAX(t.version), 0) + 1 AS version
    FROM customer_updates_type2 u
    LEFT JOIN {SILVER_SCHEMA}.customers_type2 t
        ON u.customer_id = t.customer_id
    WHERE NOT EXISTS (
        SELECT 1 FROM {SILVER_SCHEMA}.customers_type2 existing
        WHERE existing.customer_id = u.customer_id
          AND existing.is_current = true
          AND existing.city = u.city
          AND existing.email = u.email
    )
    GROUP BY u.customer_id, u.name, u.email, u.city
""")

**PO ZMIANIE (SCD Type 2 - historical tracking):**

Historia została zachowana! Wyświetlenie wszystkich wersji customer_id=1:

In [ ]:
spark.sql(f"""
    SELECT 
        customer_id,
        city,
        effective_from,
        effective_to,
        is_current,
        version
    FROM {SILVER_SCHEMA}.customers_type2 
    WHERE customer_id = 1
    ORDER BY effective_from
""").show()

**✅ Historia zachowana!**

Mamy teraz 2 rekordy:
- **Version 1**: Warszawa (effective_to = dzisiaj-1, is_current=false)  
- **Version 2**: Kraków (effective_to = 9999-12-31, is_current=true)

### Przykład: Query 'as of date'

**Gdzie mieszkał klient 1 miesiąc temu?** SCD Type 2 umożliwia temporal queries:

In [ ]:
one_month_ago = (datetime.now() - timedelta(days=30)).date()

In [ ]:
spark.sql(f"""
    SELECT 
        customer_id,
        name,
        city,
        effective_from,
        effective_to
    FROM {SILVER_SCHEMA}.customers_type2
    WHERE customer_id = 1
      AND '{one_month_ago}' BETWEEN effective_from AND effective_to
""").show()

## Sekcja 4: Gold Layer - Business Aggregates & Analytics

**Cel sekcji:** Transformacja Silver → Gold z agregacjami biznesowymi.

### Gold Layer - Kluczowe Cechy

**1. Pre-Aggregated Data**
- Daily/Monthly/Yearly summaries
- Pre-computed KPIs
- Reduced data volume (faster queries)

**2. Denormalized Tables**
- Joins pre-computed (star schema)
- Wide tables dla BI tools
- No complex joins needed

**3. Business Logic**
- Revenue calculations
- Customer segmentation
- Product performance metrics

**4. Query Optimization**
- Partitioned by report_date
- Z-ordered for common filters
- Materialized views

### Silver → Gold Transformation Pattern

**Typical Aggregation:**
```python
# Silver: Detail level (millions of rows)
silver_orders = spark.table("silver.orders_clean")

# Gold: Aggregated (thousands of rows)
gold_daily_sales = (silver_orders
    .groupBy(
        F.to_date("order_date").alias("report_date"),
        "payment_method"
    )
    .agg(
        F.count("*").alias("total_orders"),
        F.sum("total_amount").alias("total_revenue"),
        F.avg("total_amount").alias("avg_order_value"),
        F.countDistinct("customer_id").alias("unique_customers")
    )
)
```

### Gold Layer Tables - Examples

**1. Daily Sales Summary**
```sql
gold.daily_sales_summary
- report_date, payment_method
- total_orders, total_revenue, avg_order_value
- PARTITIONED BY (report_date)
```

**2. Customer 360**
```sql
gold.customer_360
- customer_id, name, email, city
- total_lifetime_value, total_orders, first_order_date, last_order_date
- customer_segment (VIP, Regular, New)
```

**3. Product Performance**
```sql
gold.product_performance
- product_id, product_name, category
- total_sold, total_revenue, avg_price
- PARTITIONED BY (category)
```

### Star Schema Pattern

**Fact Table (Orders):**
- order_id, customer_key, product_key, date_key
- total_amount, quantity

**Dimension Tables:**
- dim_customers (SCD Type 2)
- dim_products
- dim_dates

**Benefits:**
- Simplified queries
- Better performance (pre-joined)
- BI tool friendly

### Przykład 4.1: Gold - Daily Sales Summary

**Cel:** Agregacja zamówień do daily sales summary.

In [0]:
# Gold: Daily Sales Summary
silver_orders = spark.table(f"{SILVER_SCHEMA}.orders_clean")

# Aggregate to daily summary
daily_sales = (silver_orders
    .withColumn("report_date", F.to_date("order_datetime"))
    .groupBy("report_date", "payment_method")
    .agg(
        F.count("*").alias("total_orders"),
        F.sum("total_amount").alias("total_revenue"),
        F.avg("total_amount").alias("avg_order_value"),
        F.countDistinct("customer_id").alias("unique_customers"),
        F.min("total_amount").alias("min_order"),
        F.max("total_amount").alias("max_order")
    )
    .withColumn("_computation_timestamp", F.current_timestamp())
    .orderBy("report_date", "payment_method")
)

# Write to Gold
daily_sales.write.format("delta").mode("overwrite").partitionBy("report_date").saveAsTable(f"{GOLD_SCHEMA}.daily_sales_summary")

display({
    "status": f"✅ Created {GOLD_SCHEMA}.daily_sales_summary",
    "aggregated_rows": daily_sales.count(),
    "partitioned_by": "report_date"
})

**Utworzona tabela Gold:**

Tabela `daily_sales_summary` została utworzona z partycją po `report_date`. Wyświetlenie Top 10 dni po revenue:

In [ ]:
display(
    spark.table(f"{GOLD_SCHEMA}.daily_sales_summary")
    .orderBy(F.desc("total_revenue"))
    .limit(10)
)

### Data Reduction Metrics

Porównanie wielkości danych między Silver a Gold:

In [ ]:
silver_count = silver_orders.count()
gold_count = daily_sales.count()
reduction_pct = ((silver_count - gold_count) / silver_count) * 100

**Redukcja danych:**

Gold tables są znacznie mniejsze → szybsze queries!

### Przykład 4.2: Gold - Customer 360 (Denormalized)

**Cel:** Stworzenie denormalized customer view z KPIs.

In [0]:
# Gold: Customer 360
customers = spark.table(f"{SILVER_SCHEMA}.customers_type1")
orders = spark.table(f"{SILVER_SCHEMA}.orders_clean")

customer_360 = (customers
    .join(
        orders.groupBy("customer_id").agg(
            F.count("*").alias("total_orders"),
            F.sum("total_amount").alias("lifetime_value"),
            F.avg("total_amount").alias("avg_order_value"),
            F.min("order_datetime").alias("first_order_date"),
            F.max("order_datetime").alias("last_order_date")
        ),
        "customer_id",
        "left"
    )
    .withColumn("customer_segment",
        F.when(F.col("lifetime_value") > 1000, "VIP")
        .when(F.col("lifetime_value") > 500, "Regular")
        .otherwise("New")
    )
    .withColumn("_computation_timestamp", F.current_timestamp())
    .select(
        "customer_id", "name", "email", "city",
        "total_orders", "lifetime_value", "avg_order_value",
        "first_order_date", "last_order_date", "customer_segment",
        "_computation_timestamp"
    )
)

# Write to Gold
customer_360.write.format("delta").mode("overwrite").saveAsTable(f"{GOLD_SCHEMA}.customer_360")

display({
    "status": f"✅ Created {GOLD_SCHEMA}.customer_360",
    "customers": customer_360.count()
})

**Utworzona tabela Customer 360:**

Denormalized view z KPIs klientów. Top 10 klientów po Lifetime Value:

In [ ]:
display(
    spark.table(f"{GOLD_SCHEMA}.customer_360")
    .orderBy(F.desc("lifetime_value"))
    .limit(10)
)

### Customer Segmentation

Rozkład segmentów klientów:

In [ ]:
spark.sql(f"""
    SELECT 
        customer_segment,
        COUNT(*) as customer_count,
        SUM(lifetime_value) as total_value,
        AVG(lifetime_value) as avg_value
    FROM {GOLD_SCHEMA}.customer_360
    GROUP BY customer_segment
    ORDER BY total_value DESC
""").show()

**Customer 360:** Denormalized view dla BI dashboards z precomputed KPIs

## Sekcja 5: Podsumowanie & Best Practices

### Co zostało osiągnięte?

✅ **1. Medallion Architecture Implementation**
- 🥉 Bronze: Raw data landing (append-only, immutable)
- 🥈 Silver: Cleansed & validated data (deduplicated)
- 🥇 Gold: Business aggregates & KPIs (pre-computed)

✅ **2. SCD (Slowly Changing Dimensions)**
- **Type 1**: Overwrite (no history) - dla current state
- **Type 2**: Historical tracking (full history) - dla temporal analytics

✅ **3. Data Quality Patterns**
- Validation rules (NOT NULL, ranges, types)
- Quarantine pattern (valid/invalid split)
- Metadata tracking (_processing_timestamp, _is_valid)

✅ **4. Business Logic Implementations**
- Daily sales aggregations
- Customer 360 view (denormalized)
- Customer segmentation (VIP/Regular/New)

### Kluczowe Wnioski

💡 **1. Separation of Concerns**
```
Bronze = Ingestion (raw data)
Silver = Data Quality (cleansing, dedup)
Gold = Business Logic (aggregates, KPIs)
```

💡 **2. ELT > ETL**
```
Traditional ETL: Transform outside DB
Medallion ELT: Load first, transform in-place
- Preserve raw data (Bronze)
- Re-process capability
- Scalable (Spark distributed)
```

💡 **3. SCD Strategy**
```
Silver: Type 1 (current state)
Gold: Type 2 (history for dimensions)
Fact tables: Immutable (no SCD needed)
```

💡 **4. Incremental Processing**
```
Bronze → Silver: MERGE (deduplication)
Silver → Gold: Overwrite or MERGE (depends on use case)
Always use checkpoints for streaming
```

💡 **5. Partitioning Strategy**
```
Bronze: _ingestion_date
Silver: Business date (order_date, transaction_date)
Gold: Report date (report_date)
```

### Medallion Architecture - Decision Matrix

| Layer | Purpose | Schema | Updates | Retention | Use Case |
|-------|---------|--------|---------|-----------|----------|
| **Bronze** | Raw landing | Flexible | Append-only | 3-7 years | Recovery, audit |
| **Silver** | Validated | Strict | MERGE (dedup) | 1-2 years | Analytics prep |
| **Gold** | Business | Optimized | Overwrite/MERGE | 6-12 months | BI, reports |

### Production Checklist

**Bronze Layer:**
- [ ] Audit metadata (_ingestion_timestamp, _source_file)
- [ ] Schema evolution enabled (rescued_data)
- [ ] Long retention (compliance)
- [ ] Partition by _ingestion_date

**Silver Layer:**
- [ ] Data quality rules implemented
- [ ] Deduplication logic (MERGE)
- [ ] SCD Type 1 for dimension tables
- [ ] Quarantine pattern for bad data
- [ ] Partition by business date

**Gold Layer:**
- [ ] Pre-aggregated summaries
- [ ] Denormalized tables (star schema)
- [ ] SCD Type 2 for dimensions (optional)
- [ ] Partition by report_date
- [ ] Z-ordering for common filters

### Następne Kroki

**📚 Kolejne Notebooki:**
- **05_optimization_best_practices.ipynb** - Performance tuning
- **Warsztaty praktyczne** - End-to-end pipeline implementation

**🛠️ Zadanie Domowe:**
1. Zaimplementuj complete Bronze→Silver→Gold pipeline
2. Dodaj SCD Type 2 dla products dimension
3. Stwórz Gold table: monthly_product_performance
4. Zaimplementuj data quality monitoring

### Useful SQL Queries

**Query current customers only (SCD Type 2):**
```sql
SELECT * FROM silver.customers_type2
WHERE is_current = true
```

**Query historical data (as of date):**
```sql
SELECT * FROM silver.customers_type2
WHERE '2024-01-15' BETWEEN effective_from AND effective_to
```

**Gold aggregation refresh:**
```sql
INSERT OVERWRITE gold.daily_sales_summary
SELECT 
    CAST(order_date AS DATE) as report_date,
    payment_method,
    COUNT(*) as total_orders,
    SUM(total_amount) as total_revenue,
    AVG(total_amount) as avg_order_value
FROM silver.orders_clean
GROUP BY 1, 2
```

---

**Gratulacje!** 🎉 
Ukończyłeś implementację Medallion Architecture z SCD Type 1/2!
Jesteś gotowy do budowania production-grade data lakehouse pipelines!

## Sekcja 6: Czyszczenie Zasobów

**Uwaga:** Ta sekcja jest opcjonalna. Uruchom tylko jeśli chcesz usunąć wszystkie dane utworzone w tym notebooku.

### Opcja 1: Sprawdź utworzone zasoby (zalecane)

In [0]:
# Sprawdź utworzone zasoby
medallion_tables = {
    "Silver": [
        f"{SILVER_SCHEMA}.orders_clean",
        f"{SILVER_SCHEMA}.customers_type1",
        f"{SILVER_SCHEMA}.customers_type2"
    ],
    "Gold": [
        f"{GOLD_SCHEMA}.daily_sales_summary",
        f"{GOLD_SCHEMA}.customer_360"
    ]
}

results = []
for layer, tables in medallion_tables.items():
    for table in tables:
        full_table = f"{CATALOG}.{table}"
        if spark.catalog.tableExists(full_table):
            count = spark.table(full_table).count()
            detail = spark.sql(f"DESCRIBE DETAIL {full_table}").collect()[0]
            size_mb = detail['sizeInBytes'] / (1024 * 1024)
            results.append({"layer": layer, "table": table, "records": count, "size_mb": round(size_mb, 2)})

display(spark.createDataFrame(results))

**Dane są zachowane dla dalszego użytku**

Aby usunąć wszystkie tabele, uruchom następną komórkę w sekcji opcjonalnej.

### Opcja 2: Usuń wszystkie zasoby (tylko jeśli naprawdę chcesz)

**UWAGA:** To usunie wszystkie tabele Silver i Gold utworzone w tym notebooku!

In [0]:
# Opcja 2: Usuń wszystkie zasoby (TYLKO JEŚLI JESTEŚ PEWIEN!)

# ⚠️  UWAGA: Odkomentuj poniższy kod tylko jeśli chcesz usunąć wszystko!

"""
print("=== 🗑️  USUWANIE ZASOBÓW MEDALLION ===\n")

# Lista tabel do usunięcia
tables_to_drop = [
    f"{SILVER_SCHEMA}.orders_clean",
    f"{SILVER_SCHEMA}.customers_type1",
    f"{SILVER_SCHEMA}.customers_type2",
    f"{GOLD_SCHEMA}.daily_sales_summary",
    f"{GOLD_SCHEMA}.customer_360"
]

print("Usuwanie tabel...\n")
for table in tables_to_drop:
    full_table = f"{CATALOG}.{table}"
    try:
        spark.sql(f"DROP TABLE IF EXISTS {full_table}")
        print(f"  ✓ Usunięto: {table}")
    except Exception as e:
        print(f"  ⚠️  Błąd przy {table}: {e}")

print("\n✅ Czyszczenie zakończone!")
print("💡 Wszystkie tabele Medallion zostały usunięte")
print("💡 Możesz uruchomić notebook od nowa")
"""

print("⚠️  KOD CZYSZCZENIA JEST ZAKOMENTOWANY")
print("⚠️  Odkomentuj powyższy kod tylko jeśli chcesz usunąć wszystkie zasoby")
print("\n💡 Zalecenie: Zostaw dane dla kolejnych notebooków i warsztatów!")
print("💡 Następny notebook: 05_optimization_best_practices.ipynb")